<a href="https://colab.research.google.com/github/BrotherKim/Colab/blob/main/SET531/KoBERT_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#KoBERT tutorial#
 - 출처 : https://tech-diary.tistory.com/31

##KoBERT란##
 - 간단하게 말하자면, 사전에 학습된 대용량 말뭉치 모델이라고 할 수 있습니다.
 - 2018년에 위대한 구글에서 개발한 언어 모델인데, NLP 전반적인 분야에 아주 좋은 성능을 보여주는 모델이라고 합니다.
 - 그리고 이런 BERT 모델을 한국어 기반으로 제작한 것이 바로 KoBERT 입니다.

##0. KoBERT 세팅##

In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3
!pip install torch

     |████████████████████████████████| 46.9 MB 78 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 5.1 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595719 sha256=f457b37b09e34a868291044987f1e17eb3c551fdf7708bfafac443a22bd6d4cd
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 5.2 MB/s 
     |████████████████████████████████| 754 kB 6.2 MB/s 
     |████████████████████████████████| 3.0 MB 43.6 MB/s 
     |████████████████████████████████| 895 kB 33.9 MB/s 


In [2]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-hz0x77uj
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-hz0x77uj
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=12771 sha256=2e138fb9e12de177ac6ab5543e77e94e4e93985d90d912f74e109d971f35d1ab
  Stored in directory: /tmp/pip-ephem-wheel-cache-m6sr8iu5/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert


In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [4]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [5]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [6]:
##GPU 사용 시
device = torch.device("cuda:0")

In [8]:
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


##1. 데이터파일 가져오기##

In [9]:
!wget https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
!wget https://www.dropbox.com/s/977gbwh542gdy94/ratings_test.txt?dl=1

--2021-10-03 06:44:06--  https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/374ftkec978br3d/ratings_train.txt [following]
--2021-10-03 06:44:06--  https://www.dropbox.com/s/dl/374ftkec978br3d/ratings_train.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc490d95fa5e66845245c91a064e.dl.dropboxusercontent.com/cd/0/get/BXVg96dnRYRAWXTNPNxZscXANZLlk8A_qA64GKigbd0p-bnhU_jkmBdYOn3ib2oltdmdTmaZn9c1nHfjRPr0jdR383ew0G7y4I_-UxoMV5Qlb9pYZ1SYXDyaEQmMRjC8aCATN2dZJ6rIqMEhSMQ7TIdz/file?dl=1# [following]
--2021-10-03 06:44:07--  https://uc490d95fa5e66845245c91a064e.dl.dropboxusercontent.com/cd/0/get/BXVg96dnRYRAWXTNPNxZscXANZLlk8A_qA64GKigbd0p-bnhU_jkmBdYOn3ib2oltdmdTmaZn

In [10]:
dataset_train = nlp.data.TSVDataset("ratings_train.txt?dl=1", field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("ratings_test.txt?dl=1", field_indices=[1,2], num_discard_samples=1)

In [11]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [12]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


##2. 파라미터 설정##

In [13]:
## Setting parameters
max_len = 64 # 텍스트 데이터 최대 길이
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5 # 훈련 반복 횟수
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [14]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [15]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [16]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2, # 여기서는 긍정, 부정의 분류
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [17]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [18]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [19]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [20]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [21]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [22]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [23]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/2344 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 0.7185158133506775 train acc 0.5
epoch 1 batch id 201 loss 0.4861658215522766 train acc 0.5577580845771144
epoch 1 batch id 401 loss 0.4488970935344696 train acc 0.6687188279301746
epoch 1 batch id 601 loss 0.4293877184391022 train acc 0.7225457570715474
epoch 1 batch id 801 loss 0.4294663369655609 train acc 0.7535112359550562
epoch 1 batch id 1001 loss 0.28055015206336975 train acc 0.7714473026973027
epoch 1 batch id 1201 loss 0.3126758337020874 train acc 0.7856083472106578
epoch 1 batch id 1401 loss 0.3787497580051422 train acc 0.7957708779443254
epoch 1 batch id 1601 loss 0.29961809515953064 train acc 0.8043800749531543
epoch 1 batch id 1801 loss 0.27242204546928406 train acc 0.8117625624652971
epoch 1 batch id 2001 loss 0.36287620663642883 train acc 0.8176849075462269
epoch 1 batch id 2201 loss 0.34864377975463867 train acc 0.8228575079509314
epoch 1 train acc 0.8264629550625712


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 4 test acc 0.9000359654731458


  0%|          | 0/2344 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.3220340609550476 train acc 0.890625
epoch 5 batch id 201 loss 0.044367700815200806 train acc 0.972714552238806
epoch 5 batch id 401 loss 0.018437989056110382 train acc 0.9746726932668329
epoch 5 batch id 601 loss 0.12419319897890091 train acc 0.9745216306156406
epoch 5 batch id 801 loss 0.15540646016597748 train acc 0.9755774032459426
epoch 5 batch id 1001 loss 0.07442358136177063 train acc 0.9761176323676324
epoch 5 batch id 1201 loss 0.022164348512887955 train acc 0.9770503746877602
epoch 5 batch id 1401 loss 0.03362186998128891 train acc 0.9776610456816559
epoch 5 batch id 1601 loss 0.07907586544752121 train acc 0.9778946752029981
epoch 5 batch id 1801 loss 0.009283246472477913 train acc 0.9783713908939478
epoch 5 batch id 2001 loss 0.03138201683759689 train acc 0.9785497876061969
epoch 5 batch id 2201 loss 0.030020253732800484 train acc 0.9784898909586551
epoch 5 train acc 0.9786600540386803


  0%|          | 0/782 [00:00<?, ?it/s]

epoch 5 test acc 0.8998761189258312


##3. 데이터 예측##

In [24]:
import pandas as pd

# 위에서 설정한 tok, max_len, batch_size, device를 그대로 입력
# comment : 예측하고자 하는 텍스트 데이터 리스트
def getSentimentValue(comment, tok, max_len, batch_size, device):
  commnetslist = [] # 텍스트 데이터를 담을 리스트
  emo_list = [] # 감성 값을 담을 리스트
  for c in comment: # 모든 댓글
    commnetslist.append( [c, 5] ) # [댓글, 임의의 양의 정수값] 설정
    
  pdData = pd.DataFrame( commnetslist, columns = [['댓글', '감성']] )
  pdData = pdData.values
  test_set = BERTDataset(pdData, 0, 1, tok, max_len, True, False) 
  test_input = torch.utils.data.DataLoader(test_set, batch_size=batch_size, num_workers=5)
  
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_input):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length 
    # 이때, out이 예측 결과 리스트
    out = model(token_ids, valid_length, segment_ids)
	
    # e는 2가지 실수 값으로 구성된 리스트
    # 0번 인덱스가 더 크면 부정, 긍정은 반대
    for e in out:
      if e[0]>e[1]: # 부정
        value = 0
      else: #긍정
        value = 1
      emo_list.append(value)


In [ ]:
getSentimentValue